# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
weather_data = "Output_Data/new_cities.csv"

weather_df = pd.read_csv(weather_data)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,thompson,55.74,-97.86,10.40,92,90,9.17,CA,1609626614
1,ushuaia,-54.80,-68.30,55.40,66,40,21.92,AR,1609626685
2,bethel,41.37,-73.41,42.80,60,1,4.70,US,1609626724
3,busselton,-33.65,115.33,62.01,68,93,5.99,AU,1609626708
4,cape town,-33.93,18.42,69.80,60,0,21.92,ZA,1609626592


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [85]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df['Humidity'].astype(float)

fig = gmaps.figure(center=(10, 0.1), zoom_level=1.8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df['Humidity']),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [78]:
ideal_weather = weather_df.loc[(weather_df['Max Temp'] > 60) & (weather_df['Max Temp'] < 70) & (weather_df['Wind Speed'] < 5) & (weather_df['Cloudiness'] < 10)]
hotel_df = ideal_weather.dropna()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,george,-33.96,22.46,65.08,85,3,2.86,ZA,1609627061
16,champasak,14.75,106.00,61.95,84,7,1.39,LA,1609627063
59,eagle pass,28.71,-100.50,64.40,48,5,2.26,US,1609627072
61,port alfred,-33.59,26.89,64.99,86,6,3.00,ZA,1609626688
239,blythe,33.61,-114.60,64.40,22,1,1.03,US,1609627116
315,matamoros,25.53,-103.25,60.80,31,5,2.73,MX,1609626730
320,graaff-reinet,-32.25,24.53,69.01,64,0,3.78,ZA,1609626770


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
hotels = [''] * len(hotel_df.index)
hotel_df.insert(9, 'Hotel Name', hotels, True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,george,-33.96,22.46,65.08,85,3,2.86,ZA,1609627061,
16,champasak,14.75,106.00,61.95,84,7,1.39,LA,1609627063,
59,eagle pass,28.71,-100.50,64.40,48,5,2.26,US,1609627072,
61,port alfred,-33.59,26.89,64.99,86,6,3.00,ZA,1609626688,
239,blythe,33.61,-114.60,64.40,22,1,1.03,US,1609627116,
315,matamoros,25.53,-103.25,60.80,31,5,2.73,MX,1609626730,
320,graaff-reinet,-32.25,24.53,69.01,64,0,3.78,ZA,1609626770,


In [80]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius': 5000,
    "type": "lodging",
    "key": g_key,
}

hotel_coord = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row['Lat']
    lon = row['Lng']
    city = row['City']

    # add keyword to params dict
    params['location'] = f'{lat},{lon}'

    # assemble url and make API request
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_coord.append((response['results'][0]['geometry']['location']['lat'], response['results'][0]['geometry']['location']['lng']))
    
    except:
        print(f'No hotel within 5000 meters of {city}')
        hotel_df.loc[index, 'Hotel Name'] = "None within 5000 meters"


In [81]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,george,-33.96,22.46,65.08,85,3,2.86,ZA,1609627061,Protea Hotel by Marriott George King George
16,champasak,14.75,106.00,61.95,84,7,1.39,LA,1609627063,Kiat Ngong Wetland BUNGALOW
59,eagle pass,28.71,-100.50,64.40,48,5,2.26,US,1609627072,Holiday Inn Express & Suites Eagle Pass
61,port alfred,-33.59,26.89,64.99,86,6,3.00,ZA,1609626688,The Halyards Hotel
239,blythe,33.61,-114.60,64.40,22,1,1.03,US,1609627116,Rodeway Inn & Suites
315,matamoros,25.53,-103.25,60.80,31,5,2.73,MX,1609626730,Recreativo Sabdy Briseidy
320,graaff-reinet,-32.25,24.53,69.01,64,0,3.78,ZA,1609626770,Rietjiesbos B&B


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# Assign the marker layer to a variable

markers = gmaps.marker_layer(hotel_coord)

# Add the layer to the map

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))